In [1]:
from collections import defaultdict
from enum import IntEnum
from itertools import count
import intcode

with open('inputs/15.txt') as file:
    input = file.read()


In [2]:
class Cell(IntEnum):
    wall = 0
    empty = 1
    goal = 2
    air = 3
    robot = 4
    path = 5
    unknown = -1

def walk():
    prog = intcode.parse(input)
    proc = intcode.run(prog)
    f = defaultdict(lambda:Cell.unknown)
    p,d = 0,0
    dc = [1,4,2,3]
    dp = [-1j,1,1j,-1]

    while 1:
        np = p + dp[d]
        res = next(proc)
        assert res == None, "Expecting input"
        res = proc.send(dc[d])
        f[np] = res
        if np == 0:
            return f
        if res == Cell.wall:
            d = (d+1)%4
        else:
            p = np
            d = (d+3)%4

def tlbr(f):
    t,l,b,r = 0,0,0,0
    for p in f:
        t = min(t,int(p.imag))
        l = min(l,int(p.real))
        b = max(b,int(p.imag))
        r = max(r,int(p.real))
    return t,l,b,r

def shortest(f):
    f = f.copy()
    nxt = [0]
    dist = defaultdict(lambda:float('inf'))
    dp = [-1j,1,1j,-1]
    for step in count():
        if len(nxt)==0: break
        cur = nxt
        nxt = []
        for p in cur:
            if dist[p]<=step: continue
            if f[p] == Cell.goal:
                backtrack(f,dist,p)
                return step, f
            dist[p] = step
            for d in dp:
                if f[p+d] != Cell.wall: nxt.append(p+d)

def backtrack(f,dist,p):
    dp = [-1j,1,1j,-1]
    while dist[p]!=0:
        if f[p] == Cell.empty: f[p] = Cell.path
        for d in dp:
            if dist[p+d]<dist[p]:
                p = p+d
                break

def pretty(f):
    t,l,b,r = tlbr(f)
    print('')

    sym = '⬜⚫⭐➰♿⭕⛔'
    for y in range(t,b+1):
        row = ''
        for x in range(l,r+1):
            i = Cell.robot if x==0 and y==0 else f[complex(x,y)]
            row += sym[i]
        print(row)


In [3]:
%%time

f = walk()
steps, walked = shortest(f)
print(steps)
pretty(walked)


220

⛔⬜⬜⬜⬜⬜⛔⬜⬜⬜⬜⬜⛔⬜⬜⬜⛔⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⛔⬜⬜⬜⬜⬜⬜⬜⬜⬜⛔
⬜⚫⚫⚫⚫⚫⬜⚫⚫⚫⚫⚫⬜⚫⚫⚫⬜⚫⚫⚫⚫⚫⚫⚫⚫⚫⚫⚫⚫⚫⬜⚫⚫⚫⚫⚫⚫⚫⚫⚫⬜
⛔⬜⬜⚫⬜⚫⬜⬜⬜⚫⬜⚫⬜⚫⬜⚫⬜⚫⬜⬜⬜⬜⬜⚫⬜⬜⬜⬜⬜⚫⬜⬜⬜⚫⬜⬜⬜⬜⬜⚫⬜
⬜⚫⚫⚫⬜⚫⚫⚫⬜⚫⬜⚫⚫⚫⬜⚫⬜⚫⬜⚫⚫⚫⬜⚫⬜⚫⚫⚫⚫⚫⬜⚫⚫⚫⬜⚫⚫⚫⚫⚫⬜
⬜⚫⬜⬜⬜⬜⬜⚫⬜⚫⬜⚫⬜⬜⬜⬜⬜⚫⬜⚫⬜⚫⬜⚫⬜⚫⬜⬜⬜⬜⬜⚫⬜⬜⬜⚫⬜⬜⬜⬜⛔
⬜⚫⬜⚫⚫⚫⬜⚫⚫⚫⬜⚫⬜⚫⚫⚫⬜⚫⚫⚫⬜⚫⬜⚫⬜⚫⚫⚫⚫⚫⬜⚫⚫⚫⬜⚫⬜⚫⚫⚫⬜
⬜⚫⬜⚫⬜⚫⬜⬜⬜⬜⬜⬜⬜⚫⬜⚫⬜⬜⬜⬜⬜⚫⬜⚫⬜⬜⬜⬜⬜⚫⬜⬜⬜⚫⬜⚫⬜⬜⬜⚫⬜
⬜⚫⚫⚫⬜⚫⚫⚫⚫⚫⚫⚫⚫⚫⬜⚫⚫⚫⚫⚫⚫⚫⬜⚫⚫⚫⚫⚫⬜⚫⚫⚫⚫⚫⬜⚫⚫⚫⬜⚫⬜
⬜⚫⬜⬜⛔⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⚫⬜⬜⬜⬜⬜⬜⛔⬜⬜⚫⬜⚫⬜
⬜⚫⚫⚫⬜⚫⚫⚫⚫⚫⬜⚫⚫⚫⚫⚫⬜⚫⬜⚫⚫⚫⚫⚫⚫⚫⬜⚫⚫⚫⬜⚫⚫⚫⬜⚫⚫⚫⬜⚫⬜
⛔⬜⬜⚫⬜⚫⬜⬜⬜⚫⬜⚫⬜⬜⬜⚫⬜⚫⬜⚫⬜⬜⬜⬜⬜⚫⬜⬜⬜⚫⬜⚫⬜⚫⬜⚫⬜⬜⬜⚫⬜
⬜⚫⚫⚫⬜⚫⚫⚫⬜⚫⚫⚫⬜⚫⬜⚫⚫⚫⬜⚫⬜⚫⚫⚫⬜⚫⚫⚫⬜⚫⬜⚫⬜⚫⚫⚫⬜⚫⚫⚫⬜
⬜⚫⬜⬜⬜⬜⬜⚫⬜⬜⬜⬜⬜⚫⬜⬜⬜⚫⬜⚫⬜⬜⬜⚫⬜⬜⬜⚫⬜⚫⬜⚫⬜⬜⬜⬜⬜⬜⬜⚫⬜
⬜⚫⚫⚫⚫⚫⚫⚫⬜⚫⚫⚫⬜⚫⚫⚫⚫⚫⬜⚫⚫⚫⬜⚫⚫⚫⚫⚫⚫⚫⬜⚫⚫⚫⬜⚫⚫⚫⚫⚫⬜
⛔⬜⬜⬜⬜⬜⬜⬜⬜⚫⬜⚫⬜⚫⬜⬜⬜⬜⛔⬜⬜⚫⬜⬜⬜⚫⬜⬜⬜⬜⬜⬜⬜⚫⬜⬜⬜⚫⬜⬜⛔
⬜⚫⚫⚫⬜⚫⚫⚫⚫⚫⬜⚫⬜⚫⬜⚫⚫⚫⬜⚫⬜⚫⚫⚫⬜⚫⬜⚫⚫⚫⚫⚫⬜⚫⚫⚫⬜⚫⚫⚫⬜
⬜⚫⬜⬜⬜⚫⬜⬜⬜⬜⬜⚫⬜⚫⬜⚫⬜⚫⬜⚫⬜⬜⬜⚫⬜⬜⬜⚫⬜⬜⬜⚫⬜⬜⬜⚫⬜⬜⬜⚫⬜
⬜⚫⚫⚫⚫⚫⬜⚫⬜⚫⚫⚫⬜⚫⚫⚫⬜⚫⚫⚫⚫⚫⬜⚫⚫⚫⚫⚫⬜⚫⬜⚫⬜⚫⚫⚫⚫⚫⚫⚫⬜
⬜⚫⬜⬜⬜⬜⬜⚫⬜⚫⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⚫⬜⬜⬜⬜⬜⬜⬜⚫⬜⚫⬜⬜⬜⬜⬜⬜⬜⚫⬜
⬜⚫⬜⚫⚫⚫⚫⚫⬜⚫⚫⚫⚫⚫⚫⚫⚫⚫⬜⚫⚫⚫⚫⚫⚫⚫⬜⚫⚫⚫⬜⚫⚫⚫⚫⚫⬜⚫⚫⚫⬜
⬜⚫⬜⚫⬜⬜⬜⚫⬜⬜⬜⬜⬜⚫⬜⚫⬜⬜⬜⚫⬜⬜⬜⬜⬜⚫⬜⬜⬜⚫⬜⬜⬜⬜⬜⚫⬜⬜⬜⬜⛔
⬜⚫⚫⚫⬜⚫⚫⚫⬜⭕⭕⭕⬜⚫⬜⚫⬜⚫⚫⚫⬜♿⭕⭕⬜⚫⚫⚫⚫⚫⚫⚫⚫⚫⬜⚫⬜⭕⭕⭕⬜
⛔⬜⬜⬜⬜⚫⬜⬜⬜⭕⬜⭕⬜⚫⬜⬜⬜⚫⬜⬜⬜⬜⬜⭕⬜⬜⬜⚫⬜⬜⬜⬜⬜⬜⬜⚫⬜⭕⬜⭕⬜
⬜⚫⚫⚫⚫⚫⬜⭕⭕⭕⬜⭕⬜⚫⚫⚫⚫⚫⚫⚫⚫⚫⬜⭕⬜⚫⚫⚫⬜

In [4]:
def oxygenate(f):
    f = f.copy()
    tofill = 0
    for p in f:
        if f[p] == Cell.empty:
            tofill += 1
        elif f[p] == Cell.goal:
            start = p
        
    nxt = [start]
    dp = [-1j,1,1j,-1]
    for step in count(1):
        if len(nxt)==0: break
        cur = nxt
        nxt = []
        for p in cur:
            for d in dp:
                if f[p+d] != Cell.empty: continue
                f[p+d] = Cell.air
                tofill -= 1
                if tofill == 0:
                    return step, f
                nxt.append(p+d)
    

In [5]:
%%time

steps, filled = oxygenate(f)
print(steps)
pretty(filled)


334

⛔⬜⬜⬜⬜⬜⛔⬜⬜⬜⬜⬜⛔⬜⬜⬜⛔⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⛔⬜⬜⬜⬜⬜⬜⬜⬜⬜⛔
⬜➰➰➰➰➰⬜➰➰➰➰➰⬜➰➰➰⬜➰➰➰➰➰➰➰➰➰➰➰➰➰⬜➰➰➰➰➰➰➰➰➰⬜
⛔⬜⬜➰⬜➰⬜⬜⬜➰⬜➰⬜➰⬜➰⬜➰⬜⬜⬜⬜⬜➰⬜⬜⬜⬜⬜➰⬜⬜⬜➰⬜⬜⬜⬜⬜➰⬜
⬜➰➰➰⬜➰➰➰⬜➰⬜➰➰➰⬜➰⬜➰⬜➰➰➰⬜➰⬜➰➰➰➰➰⬜➰➰➰⬜➰➰➰➰➰⬜
⬜➰⬜⬜⬜⬜⬜➰⬜➰⬜➰⬜⬜⬜⬜⬜➰⬜➰⬜➰⬜➰⬜➰⬜⬜⬜⬜⬜➰⬜⬜⬜➰⬜⬜⬜⬜⛔
⬜➰⬜➰➰➰⬜➰➰➰⬜➰⬜➰➰➰⬜➰➰➰⬜➰⬜➰⬜➰➰➰➰➰⬜➰➰➰⬜➰⬜➰➰➰⬜
⬜➰⬜➰⬜➰⬜⬜⬜⬜⬜⬜⬜➰⬜➰⬜⬜⬜⬜⬜➰⬜➰⬜⬜⬜⬜⬜➰⬜⬜⬜➰⬜➰⬜⬜⬜➰⬜
⬜➰➰➰⬜➰➰➰➰➰➰➰➰➰⬜➰➰➰➰➰➰➰⬜➰➰➰➰➰⬜➰➰➰➰➰⬜➰➰➰⬜➰⬜
⬜➰⬜⬜⛔⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜➰⬜⬜⬜⬜⬜⬜⛔⬜⬜➰⬜➰⬜
⬜➰➰➰⬜➰➰➰➰➰⬜➰➰➰➰➰⬜➰⬜➰➰➰➰➰➰➰⬜➰➰➰⬜➰➰➰⬜➰➰➰⬜➰⬜
⛔⬜⬜➰⬜➰⬜⬜⬜➰⬜➰⬜⬜⬜➰⬜➰⬜➰⬜⬜⬜⬜⬜➰⬜⬜⬜➰⬜➰⬜➰⬜➰⬜⬜⬜➰⬜
⬜➰➰➰⬜➰➰➰⬜➰➰➰⬜➰⬜➰➰➰⬜➰⬜➰➰➰⬜➰➰➰⬜➰⬜➰⬜➰➰➰⬜➰➰➰⬜
⬜➰⬜⬜⬜⬜⬜➰⬜⬜⬜⬜⬜➰⬜⬜⬜➰⬜➰⬜⬜⬜➰⬜⬜⬜➰⬜➰⬜➰⬜⬜⬜⬜⬜⬜⬜➰⬜
⬜➰➰➰➰➰➰➰⬜➰➰➰⬜➰➰➰➰➰⬜➰➰➰⬜➰➰➰➰➰➰➰⬜➰➰➰⬜➰➰➰➰➰⬜
⛔⬜⬜⬜⬜⬜⬜⬜⬜➰⬜➰⬜➰⬜⬜⬜⬜⛔⬜⬜➰⬜⬜⬜➰⬜⬜⬜⬜⬜⬜⬜➰⬜⬜⬜➰⬜⬜⛔
⬜➰➰➰⬜➰➰➰➰➰⬜➰⬜➰⬜➰➰➰⬜➰⬜➰➰➰⬜➰⬜➰➰➰➰➰⬜➰➰➰⬜➰➰➰⬜
⬜➰⬜⬜⬜➰⬜⬜⬜⬜⬜➰⬜➰⬜➰⬜➰⬜➰⬜⬜⬜➰⬜⬜⬜➰⬜⬜⬜➰⬜⬜⬜➰⬜⬜⬜➰⬜
⬜➰➰➰➰➰⬜➰⬜➰➰➰⬜➰➰➰⬜➰➰➰➰➰⬜➰➰➰➰➰⬜➰⬜➰⬜➰➰➰➰➰➰➰⬜
⬜➰⬜⬜⬜⬜⬜➰⬜➰⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜➰⬜⬜⬜⬜⬜⬜⬜➰⬜➰⬜⬜⬜⬜⬜⬜⬜➰⬜
⬜➰⬜➰➰➰➰➰⬜➰➰➰➰➰➰➰➰➰⬜➰➰➰➰➰➰➰⬜➰➰➰⬜➰➰➰➰➰⬜➰➰➰⬜
⬜➰⬜➰⬜⬜⬜➰⬜⬜⬜⬜⬜➰⬜➰⬜⬜⬜➰⬜⬜⬜⬜⬜➰⬜⬜⬜➰⬜⬜⬜⬜⬜➰⬜⬜⬜⬜⛔
⬜➰➰➰⬜➰➰➰⬜➰➰➰⬜➰⬜➰⬜➰➰➰⬜♿➰➰⬜➰➰➰➰➰➰➰➰➰⬜➰⬜➰➰➰⬜
⛔⬜⬜⬜⬜➰⬜⬜⬜➰⬜➰⬜➰⬜⬜⬜➰⬜⬜⬜⬜⬜➰⬜⬜⬜➰⬜⬜⬜⬜⬜⬜⬜➰⬜➰⬜➰⬜
⬜➰➰➰➰➰⬜➰➰➰⬜➰⬜➰➰➰➰➰➰➰➰➰⬜➰⬜➰➰➰⬜